<a href="https://colab.research.google.com/github/yongyogn/practice/blob/prac/220704.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 데이터 실수화
import pandas as pd

# 데이터셋 생성, 확인
x_train=pd.DataFrame(['남성','여성','남성','여성','남성','여성'],columns=['성별'])
print('#x_train.info():\n',x_train.info())
print('\n#x_train["성별"].value_counts():\n',x_train['성별'].value_counts())


# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
encoding=LabelEncoder()
encoding.fit(x_train['성별'])
x_train['성별_인코딩']=encoding.transform(x_train['성별'])
x_train
print("\n클래스\n",list(encoding.classes_)) # 클래스 확인
print("\n인코딩 값으로 문자값 확인:\n",list(encoding.inverse_transform([1,0]))) # 인코딩이 1로 된 문자(여자), 0으로 된 문자(남자)
print(x_train)

# 원핫 인코딩
from sklearn.preprocessing import OneHotEncoder
one_encoding=OneHotEncoder(sparse=False) # sparse가 True인 경우 "(행,열) 1"의 좌료 리스트 형식, False인 경우 numpy 배열로 반환
one_encoding.fit(x_train[['성별']]) # x_train 데이터를 인코더에 피팅
one_encoding.transform(x_train[['성별']]) # x_train 데이터에 피팅된 인코더로 x_train 데이터를 가변환값으로 변환
print("\n가변화된 피쳐확인:\n",one_encoding.get_feature_names_out())

x_train_one=pd.DataFrame(one_encoding.transform(x_train[['성별']]),columns=['성별0','성별1']) # 가변화값을 데이터프레임으로 저장하며 열이름은 성별0, 성별1로 변환
x_train=pd.concat([x_train,x_train_one],axis=1) # x_train 데이터와 가변화된 값 합침
print(x_train)

# 단어의 빈도수를 카운트하여 단어를 수치화하는 방법
from sklearn.feature_extraction.text import CountVectorizer
corpus=[
            '청년 인재 개발 양성 과정',
            '인공지능 청년 양성',
            '미래 인공지능 데이터 대한민국',
            '데이터 원유 기술사 청년 개발'
        ]
print(corpus)
print(type(corpus))
count_vect=CountVectorizer()
count_vect.fit(corpus)
count_vect.transform(corpus)
print(count_vect.vocabulary_)
count_vect.transform(corpus).toarray() # 반어 벡터화한 값을 array로 변환하여 확인
features=count_vect.transform(corpus)
print(features)
vocab=count_vect.get_feature_names_out() # 속성의 이름만 반환
print(vocab)
DTM=pd.DataFrame(features.toarray(),columns=vocab) # 문서 단어 행렬을 데이터프레임 형태로 변환
print(DTM)
count_vect.transform(['기술사 대한민국 인재 만세']).toarray()

# 단어의 빈도수를 단어의 중요도를 가중치로 두어 수치화하는 방법
from sklearn.feature_extraction.text import TfidfVectorizer
tfid=TfidfVectorizer()
tfid.fit(corpus)
print(tfid.vocabulary_) # 단어사전 확인
tfid.transform(corpus).toarray() # 단어 벡터화한 값을 array로 변환하여 확인
features_idf=tfid.transform(corpus)
print(features_idf)
vocab_idf=count_vect.get_feature_names_out()
print(vocab_idf)
DTM_idf=pd.DataFrame(features_idf.toarray(),columns=vocab_idf).head()
DTM_idf

In [ ]:
# 결측치 처리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
if os.name=='nt':
    font_family='Malgun Gothic'
else:
    font_family='AppleGothic'
sns.set(font=font_family,rc={"axes.unicode_minus":False})

temp_dict={'A열':[np.nan,4,6,5,5,np.nan],
           'B열':['a','a',np.nan,'b','b','b'],
           'C열':[np.nan,3.5,-1.5,np.nan,5.5,np.nan],
           'D열':[1,0,1,0,1,1]
          }
missing_data=pd.DataFrame(temp_dict)
print("A열 결측값 개수: ",missing_data['A열'].isnull().sum())
print("B열 결측값 개수: ",missing_data['B열'].isnull().sum())
print("C열 결측값 개수: ",missing_data['C열'].isnull().sum())
print("D열 결측값 개수: ",missing_data['D열'].isnull().sum())
missing_data.isnull().sum().sort_values().plot.barh(figsize=(10,5)) # 각 열의 결측값 개수 가로막대그래프로 표현

# 결측치 처리-삭제
missing_data.dropna(axis=0) # 결측값이 있는 행 삭제
missing_data.dropna(axis=1) # 결측값이 있는 열 삭제
missing_data.dropna(thresh=5,axis=1) # 결측값을 제외한 뒤 개수가 5개 미만인 열 삭제
missing_data[missing_data['B열'].notnull()] # 특정 열(B열)에 결측값이 존재하는 행 삭제

# 결측치 처리-득정값으로 채우기
missing_data['A열'].fillna(0) # 결측치를 특정 값(0)으로 채우기
missing_data['B열'].fillna(method='ffill') # 결측치를 이전 값으로 채우기
missing_data['B열'].fillna(method='bfill') # 결측치를 이후 값으로 채우기
missing_data.fillna(missing_data.mean()) # 결측치를 속성별 평균값으로 채우기

# 결측치 처리-대체
from sklearn.impute import SimpleImputer
df=missing_data.copy()
imputer=SimpleImputer(strategy='most_frequent') # 최빈값으로 대체
df['A열']=imputer.fit_transform(missing_data[['A열']]) # A열의 결측치를 A열의 최빈값으로 대체
df=pd.DataFrame(imputer.fit_transform(df),columns=df.columns) # 나머지 열의 결측치도 각 열의 최빈값으로 대체

# 결측치 처리-선형회귀법을 이용해 나온 값으로 대체
from sklearn import linear_model
data=pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data",header=None, names=['sex','length','diameter','height','whole_weight','shucked_weight','viscera_weight','shell_weight','rings'])
df_missing=data.copy()[:20] # 불러온 데이터 중 20개만 사용
df_missing.loc[0:4,'whole_weight']=np.nan # whole_weight 열의 앞의 5개의 데이터를 임의로 결측치로 변경
X=df_missing.dropna(axis=0)[['diameter','height','shell_weight']] # X에 3개의 변수 할당하면서 결측치 있는 행은 삭제
y=df_missing.dropna(axis=0)['whole_weight'] # y에 whole_weight 변수 할당하면서 결측치 있는 행은 삭제
lin_reg=linear_model.LinearRegression()
lin_reg_model=lin_reg.fit(X,y) # X와 y를 각각 독립변수와 종속변수로하여 회귀식 피팅
y_pred=lin_reg_model.predict(df_missing.loc[:,['diameter','height','shell_weight']]) # 회귀식에 독립변수 대입해서 y 값 예측
df_missing['whole_weight'].fillna(pd.Series(y_pred.flatten()),inplace=True) # 회귀식을 통해 예측된 y값으로 결측치를 예측하여 대체
# flatten(): 다차원 배열 공간을 1차원으로 평탄화
df_missing.head()

In [ ]:
# 이상값 처리
import random
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
if os.name=='nt':
    font_family='Malgun Gothic'
else:
    font_family='AppleGothic'
sns.set(font=font_family,rc={"axes.unicode_minus":False})

np_data=np.random.randint(1,100,size=3000)
data=list(np_data)
plt.hist(data) # 히스토그램으로 데이터의 분포 확인
plt.title("데이터 분포")
plt.show()
data_mean=np.mean(np_data)
data_std=np.std(np_data)
print("평균:",data_mean)
print("표준편차:",data_std)
pdf=stats.norm.pdf(np.sort(data),data_mean,data_std) # 확률밀도함수 생성
plt.figure()
plt.plot(np.sort(data),pdf)
plt.title("데이터의 확률밀도함수")

# 표준정규분포로 변환
std_list=[]
for i in data:
    z=(i-data_mean)/data_std
    std_list.append(z)
ol_data_mean=np.mean(std_list)
ol_data_std=np.std(std_list)
print("표준정규분포로 변환한 데이터의 평균:",ol_data_mean)
print("표준정규분포로 변환한 데이터의 표준편차:",ol_data_std)
ol_pdf=stats.norm.pdf(np.sort(std_list),ol_data_mean,ol_data_std)
plt.figure()
plt.plot(np.sort(data),ol_pdf)
plt.title("표준정규분포로 변환한 데이터의 분포")

ol_data=data.copy()
ol_no=10
for i in range(ol_no):
    rand_no=random.randint(0,len(data)) # 0부터 3000까지 중 임의로 10개 뽑기
    ol_data.insert(rand_no,ol_data[rand_no]*2) # 뽑힌 숫자번째의 데이터 값을 두배로 하여 10개의 이상값 생성
ol_data_mean=np.mean(ol_data)
ol_data_std=np.std(ol_data)
print("이상치 포함한 데이터의 평균:",ol_data_mean)
print("이상치 포함한 데이터의 표준편차:",ol_data_std)
ol_pdf=stats.norm.pdf(np.sort(ol_data),ol_data_mean,ol_data_std)
plt.figure()
plt.plot(np.sort(ol_data),ol_pdf)
plt.title("이상치를 포함한 데이터의 분포")

# 표즌화 후 이상값 삭제
ol_std_list=[]
for i in ol_data:
    z=(i-ol_data_mean)/ol_data_std
    ol_std_list.append(z)
ol_st_data_mean=np.mean(ol_std_list)
ol_st_data_std=np.std(ol_std_list)
print("표준화 이후 평균:",ol_st_data_mean)
print("표준화 이후 표준편차:",ol_st_data_std)
ol_std_pdf=stats.norm.pdf(np.sort(ol_std_list),ol_st_data_mean,ol_st_data_std)
plt.figure()
plt.plot(np.sort(ol_std_list),ol_std_pdf)
plt.title("이상치 있는 상태의 확률밀도함수")

# z값이 2가 넘어가는 데이터 삭제
for i in ol_std_list:
    z=i
    if z>2:
        ol_std_list.remove(z)
    if z<-2:
        ol_std_list.remove(z)
len(ol_std_list) # z값이 2가 넘어가는 데이터 삭제후 데이터 개수
ol_std_pdf=stats.norm.pdf(np.sort(ol_std_list),ol_st_data_mean,ol_st_data_std)
plt.figure()
plt.title("z값이 2가 넘어가는 이상치 삭제 후 확률밀도함수")
plt.plot(np.sort(ol_std_list),ol_std_pdf)

# 사분위범위로 이상값 삭제
boxdata=pd.DataFrame(np.random.randn(100,3),columns=['A열','B열','C열'])
boxdata.describe()
sns.boxplot(data=boxdata)
plt.title("이상치 처리 전 상자그림")
plt.show()
def get_outlier(df=None,column=None,weight=1.5): # 사분위범위 구해서 1.5배한 값의 범위 밖에 있는 값을 이상값으로 정의
    quantile_25=np.percentile(df[column].values,25) # 1사분위수
    quantile_75=np.percentile(df[column].values,75) # 3사분위수
    IQR=quantile_75-quantile_25 # 사분위범위=3사분위수-1사분위수
    IQR_weight=IQR*weight
    lowest=quantile_25-IQR_weight
    highest=quantile_75+IQR_weight
    outlier_idx=df[column][(df[column]<lowest)|(df[column]>highest)].index
    return outlier_idx
outlier_idx=get_outlier(df=boxdata,column='B열',weight=1.5)
outlier_idx
boxdata.drop(outlier_idx,axis=0,inplace=True) # 이상값 있는 행 삭제
boxdata.shape
sns.boxplot(data=boxdata)
plt.title("B열 이상치 삭제 후 상자그림")
plt.show()
outlier_idx=get_outlier(df=boxdata,column='A열',weight=1.5)
boxdata.drop(outlier_idx,axis=0,inplace=True)
sns.boxplot(data=boxdata)
plt.title("A열 이상치 삭제 후 상자그림")
plt.show()

In [ ]:
# 변수선택
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
if os.name=='nt':
    font_family='Malgun Gothic'
else:
    font_family='AppleGothic'
sns.set(font=font_family,rc={"axes.unicode_minus":False})

from sklearn.datasets import load_iris
iris=load_iris()
iris.feature_names
df=pd.DataFrame(iris.data,columns=iris.feature_names)
sy=pd.Series(iris.target,dtype='category')
sy=sy.cat.rename_categories(iris.target_names)
df['species']=sy
cols=iris.feature_names

# 필터기법-분산이 임계치보다 낮은 변수 제거
from sklearn.feature_selection import VarianceThreshold
selector=VarianceThreshold()
selector.fit(df[cols]) # threshold(임계치) 설정 없이 분산 산출
print(selector.variances_)
selector.set_params(threshold=0.6) # threshold(임계치) 설정
v_sel=selector.fit_transform(df[cols])
print(selector.variances_)
print(selector.get_support()) # 분산이 임계치보다 낮은 feature 확인
iris_sel_df=df.iloc[:,[0,2]] # False(임계치보다 분산이 낮은 열)인 열 제외, True(임계치보다 분산이 높은 열)만 선택
iris_sel_df
iris_sel_df=pd.concat([iris_sel_df,df[['species']]],axis=1)
iris_sel_df

# 필터기법-큰 상관계수 이용
cor=df.corr()
sns.heatmap(cor,annot=False,cmap=plt.cm.Blues)
plt.show()
cor_target=abs(cor['sepal length (cm)'])
print(cor_target)
selected_cols=cor_target[cor_target>0.6]
print("상관계수>0.6인 속성")
print(selected_cols)
df[selected_cols.index].head()
df_cor_sel=df[selected_cols.index]
df_cor_sel.head()

# 새 섹션